In [25]:
#! /usr/local/bin/python
# -*- coding: utf-8 -*-

In [26]:
# prepare an export to a UTF-8 encoded file
import codecs
qc_out = '/Users/leila/Documents/OOI_GitHub_repo/repos/interacting_w_preload/interacting_w_qcTables/data_qc_global_range_values.csv'
fileout =  codecs.open(qc_out, 'w','utf-8')

In [27]:
# read qc-lookup table from Github
import pandas as pd
import csv
qc_url = 'https://raw.githubusercontent.com/ooi-integration/qc-lookup/master/data_qc_global_range_values.csv'
qc_file = pd.read_csv(qc_url, keep_default_na = False, quoting = csv.QUOTE_ALL)
qc_file.keys()
# all temperature parameter listed in qc sheet
qc_T_data = qc_file[qc_file['_Units'].str.contains('ºC')]
# make a unique list of parameters
qc_T_data = qc_T_data.ParameterID_T.unique()

In [28]:
# read the sheet created from the recent global ranges review for the temperature parameters 
review_qc = '/Users/leila/Documents/OOI_GitHub_repo/repos/interacting_w_preload/compare_to_preload/Temperature_GR_reviewed.csv'
data_sheet = pd.read_csv(review_qc, dtype={'data_level': str}, keep_default_na = False)
review_T_list = data_sheet.parameter_name.values.tolist()

In [29]:
# compare qc sheet with my work
diff_2_drop = set(qc_T_data) - set(review_T_list)
diff_2_drop = list(diff_2_drop)

In [30]:
# how to drop unwated parameters 
# NOTE: this no longer working because changes were pushed to github
#for i in range (len(diff_2_drop)):
#    qc_data = qc_data.drop(qc_data[(qc_data['_Units'] == 'ºC') & (qc_data['ParameterID_R'] == diff_2_drop[i])].index)

In [31]:
# compare my work with qc sheet
diff_2_keep = set(review_T_list) - set(qc_T_data)
diff_2_keep = list(diff_2_keep)
diff_2_keep

['external_temperature',
 'pco2w_thermistor_temperature',
 'dosta_analog_optode_temperature',
 'sensor_temperature']

How to add parameters to qc look-up table

In [32]:
# Step 1: get PD numbers an other attributes of the parameters to add to the qc look-up table
new_param = data_sheet.loc[data_sheet['parameter_name'].isin(diff_2_keep)]
pdnum = list(new_param.parameter_id.values)
pname2keep = list(new_param.parameter_name.values)
pGlobalRangeMin = list(new_param.GlobalRangeMin.values)
pGlobalRangeMax = list(new_param.GlobalRangeMax.values)
pdata_level = list(new_param.data_level.values)
punit = list(new_param.unit.values)

print(pdnum,pname2keep,pdata_level,new_param.keys())

([930, 3458, 3463, 3793], ['pco2w_thermistor_temperature', 'external_temperature', 'sensor_temperature', 'dosta_analog_optode_temperature'], ['1', '', '', ''], Index([u'parameter_id', u'parameter_name', u'unit', u'display_name',
       u'standard_name', u'data_product_identifier', u'parameter_description',
       u'data_level', u'data_product_type', u'GlobalRangeMin',
       u'GlobalRangeMax', u'Type'],
      dtype='object'))


In [33]:
# step 2 get stream names
# Read preload sheet to map the pd number associated with the parameters above to the stream names
import pandas as pd
import csv
github_url = 'https://raw.githubusercontent.com/oceanobservatories/preload-database/master/csv/ParameterDictionary.csv'
preload_data = pd.read_csv(github_url, keep_default_na=False, quoting=csv.QUOTE_ALL)
# drop the engineering streams, only science streams are considered
preload_data = preload_data.drop(preload_data[(preload_data['streamtype'] == 'Engineering')].index)

In [34]:
# create a dictionary
d = {'stream':[],'pdnum':[],'parameter':[],'GlobalRangeMin':[],'GlobalRangeMax':[],'data_level':[],'unit':[]}
for i in range(len(pdnum)):
    new_list = preload_data[preload_data['parameterids'].str.contains('PD' + str(pdnum[i]))]
    d['stream'].append(list(new_list.name.values))
    d['pdnum'].append(pdnum[i])
    d['parameter'].append(pname2keep[i])
    d['GlobalRangeMin'].append(pGlobalRangeMin[i])
    d['GlobalRangeMax'].append(pGlobalRangeMax[i])
    d['data_level'].append(pdata_level[i])
    d['unit'].append(punit[i])
stream_list = list(new_list.name.values)

In [35]:
#for i, (header, data) in enumerate(d.items()):
#    print header[:]
print d['parameter'][0]
print d['pdnum'][0]
print d['stream'][0]
print d['GlobalRangeMin'][0]
print d['GlobalRangeMax'][0]
print d['data_level'][0]
print d['unit'][0]

pco2w_thermistor_temperature
930
['pco2w_a_sami_data_record', 'pco2w_b_sami_data_record', 'pco2w_abc_instrument', 'pco2w_abc_instrument_blank', 'pco2w_abc_dcl_instrument', 'pco2w_abc_dcl_instrument_recovered', 'pco2w_abc_dcl_instrument_blank', 'pco2w_abc_dcl_instrument_blank_recovered', 'pco2w_abc_imodem_instrument', 'pco2w_abc_imodem_instrument_recovered', 'pco2w_abc_imodem_instrument_blank', 'pco2w_abc_imodem_instrument_blank_recovered']
-2
40
1
deg_C


In [36]:
print(d, stream_list)

({'GlobalRangeMax': ['40', '40', '40', '40'], 'pdnum': [930, 3458, 3463, 3793], 'data_level': ['1', '', '', ''], 'stream': [['pco2w_a_sami_data_record', 'pco2w_b_sami_data_record', 'pco2w_abc_instrument', 'pco2w_abc_instrument_blank', 'pco2w_abc_dcl_instrument', 'pco2w_abc_dcl_instrument_recovered', 'pco2w_abc_dcl_instrument_blank', 'pco2w_abc_dcl_instrument_blank_recovered', 'pco2w_abc_imodem_instrument', 'pco2w_abc_imodem_instrument_recovered', 'pco2w_abc_imodem_instrument_blank', 'pco2w_abc_imodem_instrument_blank_recovered'], ['dosta_ln_auv_instrument', 'dosta_ln_auv_instrument_recovered'], ['parad_n_auv_instrument', 'parad_n_auv_instrument_recovered'], ['dosta_abcdjm_ctdbp_p_dcl_instrument', 'dosta_abcdjm_ctdbp_p_dcl_instrument_recovered']], 'GlobalRangeMin': ['-2', '-2', '-2', '-2'], 'parameter': ['pco2w_thermistor_temperature', 'external_temperature', 'sensor_temperature', 'dosta_analog_optode_temperature'], 'unit': ['deg_C', 'deg_c', 'deg_C', 'deg_C']}, ['dosta_abcdjm_ctdbp_p_d

In [37]:
# step 3 get the reference designators names
# read the data team database to map the streams to the reference designators.
import pandas as pd
import csv
db_url = 'https://raw.githubusercontent.com/seagrinch/data-team-python/master/infrastructure/data_streams.csv'
db_data = pd.read_csv(db_url, keep_default_na=False, quoting=csv.QUOTE_ALL)
db_data.keys()

Index([u'reference_designator', u'method', u'stream_name', u'uframe_route',
       u'driver', u'parser', u'instrument_type'],
      dtype='object')

In [38]:
#extract list of ref des to add to qc sheet for new parameteres to add
dd = {'ReferenceDesignator':[],'ParameterID_R':[],'ParameterID_T':[],'GlobalRangeMin':[],'GlobalRangeMax':[],
      '_DataLevel':[],'_Units':[],'_Array ID':[],'_Platform ID':[],'_Instrument':[]}
for i in range(len(d['parameter'])):
    for j in range(len(d['stream'][i])):
        db_data_i = db_data.loc[(db_data['stream_name'] == d['stream'][i][j])]
        ref_list = list(db_data_i.reference_designator.values)   
        for k in range(len(ref_list)):
            dd['ReferenceDesignator'].append(ref_list[k])
            #dd['stream'].append(d['stream'][i][j])
            dd['ParameterID_R'].append(d['parameter'][i])
            dd['ParameterID_T'].append(d['parameter'][i])
            dd['GlobalRangeMin'].append(d['GlobalRangeMin'][i])
            dd['GlobalRangeMax'].append(d['GlobalRangeMax'][i])            
             
            if d['data_level'][i] == '':
                dd['_DataLevel'].append('')
                print 'not defined'
            else:
                dd['_DataLevel'].append('L' + d['data_level'][i]) 
                print('L' + d['data_level'][i])
            
            dd['_Units'].append('ºC')
            dd['_Array ID'].append(ref_list[k][0:8])
            dd['_Platform ID'].append(ref_list[k][9:14])
            dd['_Instrument'].append(ref_list[k][18:27])       

L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
L1
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined
not defined


In [41]:
dff= pd.DataFrame(dd, columns=['ReferenceDesignator','ParameterID_R','ParameterID_T',
                               'GlobalRangeMin','GlobalRangeMax','_DataLevel','_Units',
                               '_Array ID','_Platform ID','_Instrument'])
dff = dff.drop_duplicates(dff)
qc_file = qc_file.append(dff, ignore_index = False)

In [42]:
qc_file.to_csv('/Users/leila/Documents/OOI_GitHub_repo/repos/interacting_w_preload/interacting_w_qcTables/data_qc_global_range_values.csv',
                    index=False)